In [ ]:
# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [47]:
# 1. 10 countries with the lowest infant mortality rates

In [86]:
import pandas as pd
document_tree = ET.parse( './data/mondial_database.xml' )
d_mortality = {}
l_mortality = []
for element in document_tree.iterfind('country'):
    if element.find('infant_mortality') is not None:
        d_mortality[element.find('name').text] = element.find('infant_mortality').text
        l_mortality.append(d_mortality)

df_mortality = pd.DataFrame.from_dict(d_mortality.items())
df_mortality.sort(1).head(10)

,0,1
34,Monaco,1.81
101,Romania,10.16
226,Fiji,10.2
128,Brunei,10.48
157,Grenada,10.5
100,Mauritius,10.59
116,Panama,10.7
70,Seychelles,10.77
43,United Arab Emirates,10.92
175,Barbados,10.93


In [50]:
# 2. 10 cities with the largest population

In [130]:
document_tree = ET.parse( './data/mondial_database.xml' )

l_population = []

for element in document_tree.iterfind('country'):
    for subelement in element.getiterator('city'):
        for popu in subelement.iter('population'):
            d_popultion = {}
            d_popultion['country'] = element.find('name').text
            d_popultion['city'] = subelement.find('name').text
            d_popultion['year'] = int(popu.attrib['year'])
            d_popultion['population'] = int(popu.text)
            l_population.append(d_popultion)
                    
df_population = pd.DataFrame(l_population)
df_population.groupby(['city']).max().sort('population', ascending=False).head(10)

,country,population,year
city,,,
Shanghai,China,22315474,2010
Istanbul,Turkey,13710512,2012
Delhi,India,12877470,2011
Mumbai,India,12442373,2011
Moskva,Russia,11979529,2013
Beijing,China,11716620,2010
São Paulo,Brazil,11152344,2010
Tianjin,China,11090314,2010
Guangzhou,China,11071424,2010


In [118]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [204]:
document_tree = ET.parse( './data/mondial_database.xml' )

l_population = []

for element in document_tree.iterfind('country'):
    l_ethnic_percent = []  
    l_ethnic_name = [] 
    # get all the ethnic groups in this country and put them in two lists
    for ethnic in element.iter('ethnicgroup'):
        curr_percent = float(ethnic.attrib['percentage'])
        l_ethnic_percent.append(curr_percent)
        l_ethnic_name.append(ethnic.text)
    
    for popu in element.iterfind('population'):
        d_popultion = {}
        d_popultion['country'] = element.find('name').text
        d_popultion['year'] = int(popu.attrib['year'])
        #multiply the population into each of the ethnic group's percentage
        for i in range(len(l_ethnic_percent)):
            d_popultion['ethnic_population'] = float(popu.text) * l_ethnic_percent[i]
            d_popultion['ethnic_name'] = l_ethnic_name[i]
            d_popultion['ethnic_percent'] = l_ethnic_percent[i]
        l_population.append(d_popultion)
               
df_population = pd.DataFrame(l_population)
df_grouped = df_population.groupby(['country', 'ethnic_name'], as_index=False)['ethnic_population'].max()
sum_ethinic = df_grouped.groupby('ethnic_name')['ethnic_population'].sum()
df_sum = pd.DataFrame(sum_ethinic)
df_sum.sort('ethnic_population',ascending=False).head(10)


,ethnic_population
ethnic_name,
Han Chinese,1.245059e+11
African,2.488789e+10
Bengali,1.467769e+10
Japanese,1.272890e+10
Malay,1.081002e+10
Eastern Hamitic,8.283038e+09
Arab-Berber,8.006080e+09
European,7.357568e+09
Thai,4.948624e+09


In [205]:
# 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [268]:
document_tree = ET.parse( './data/mondial_database.xml' )

l_country = []
# first create a df for country and country code
for element in document_tree.iterfind('country'):
    d_country = {}
    d_country['country_name'] = element.find('name').text
    d_country['code'] = element.attrib['car_code']
    l_country.append(d_country)
        
df_country = pd.DataFrame(l_country)
        
# now get lake info   
l_lake = []
for element in document_tree.iterfind('lake'):
    d_lake = {}
    d_lake['lake_name'] = element.find('name').text
    if element.find('located') is not None:
        d_lake['located'] = element.find('located').attrib['country']
    if element.find('area') is not None:
        d_lake['area'] = float(element.find('area').text)
    l_lake.append(d_lake)
    
df_lake = pd.DataFrame(l_lake)

# now get river info
l_river = []
for element in document_tree.iterfind('river'):
    d_river = {}
    d_river['located'] = element.attrib['country']
    d_river['river_name'] = element.find('name').text
    if element.find('length') is not None:
        d_river['length'] = float(element.find('length').text)
    l_river.append(d_river)
    
df_river = pd.DataFrame(l_river)

#now get airport information
l_airport = []
for element in document_tree.iterfind('airport'):
    d_airport = {}
    d_airport['airport_name'] = element.find('name').text
    if element.find('elevation') is not None:
        if element.find('elevation').text is not None:
            d_airport['elevation'] = int(element.find('elevation').text)
    d_airport['located'] = element.attrib['country']
    l_airport.append(d_airport)
    
df_airport = pd.DataFrame(l_airport)

df_1 = df_country.merge(df_river, left_on='code', right_on='located', how='inner')

df_2 = df_1.merge(df_lake, left_on='code', right_on='located', how='inner')

df_all = df_2.merge(df_airport, left_on='code', right_on='located', how='inner')

longest_river = df_all.sort('length', ascending=False).head(1)

largest_lake = df_all.sort('area', ascending=False).head(1)

highest_airport = df_all.sort('elevation', ascending=False).head(1)

print "Longest river " + longest_river.river_name + " in " + longest_river.country_name
print "Largest lake " + largest_lake.lake_name + " in " + largest_lake.country_name
print "Highest airport " + highest_airport.airport_name + " in " + highest_airport.country_name


13821    Longest river Jangtse in China
dtype: object
6197    Largest lake Caspian Sea in Russia
dtype: object
36554    Highest airport El Alto Intl in Bolivia
dtype: object
